## Title: Assignment 2: Credit Card Fraud Detection

# Author: " 20001247 "

## Introduction:
In assignment 2, you will get familier with Jupyter Notebook and Python for predictive data analytics task. Jupyter notebook is very similar to Rmarkdown,it is Python veriosn of Rmarkdown

To start with, you should install Jupyter Notenook on your laptop/PC. The easist way for beginners is to install Jupyter Notebook via Anaconda https://www.anaconda.com/distribution/. You can also install Jupyter Notebook from Python. A nice tutorial on Jupyter Notebook installation and usage: https://www.dataquest.io/blog/jupyter-notebook-tutorial/. Introduction to the history of Jupyter Notebook: https://www.youtube.com/watch?v=ctOM-Gza04Y

Your goal in this assignment is to compare multiple machine learning classifiers, namely Logistic Regression (covered in topic 1), Random Forest (typical bagging method), XGBoost on a popular predictive data analystics task, i.e., Credit Card Fraud Detection. You will also be asked to perform hyperparameter tuning for three classifiers using random search and grid search. 

You can always add more code/markdown cells.

## Dataset
The dataset we are using is from Kaggle: https://www.kaggle.com/mlg-ulb/creditcardfraud#creditcard.csv. The datasets contains transactions made by credit cards in September 2013 by european cardholders. This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, ... V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-senstive learning. Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.

In [3]:
# import needed Python libraries, e.g., sklearn and xgboost 
from sklearn.model_selection import train_test_split, cross_val_score # for cross validation
from sklearn import datasets  # import build-in dataset
from sklearn import svm, linear_model
from sklearn.linear_model import LogisticRegression  # import model
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import imblearn as imb
import sys
# !{sys.executable} -m pip install jupyterthemes
# !{sys.executable} -m pip install xgboost
import jupyterthemes as jt
import xgboost as xgb
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
print("completed importing libraries")


completed importing libraries


## Task 1: load dataset and set up pipeline (10 points).
In reality, we usually use 10-fold cross validation rather than reporting evalution performance on one single split of data. Cross-validation: Ref. https://towardsdatascience.com/5-reasons-why-you-should-use-cross-validation-in-your-data-science-project-8163311a1e79 

Using pandas to load and explore data, e.g., dataset = pd.read_csv(r"D:/Datasets/creditfraud.csv", sep=';'). 
Ref: https://www.datacamp.com/community/tutorials/pandas-read-csv

pandas is similar to dataframe in R. You can also use dataset.head(3) to check the initial 3 rows of your dataset.

Using scikit-learn package, you can easily build data analytics pipeline.
e.g., 
``` 
from sklearn.model_selection import cross_val_score # for cross validation
from sklearn import datasets  # import build-in dataset
from sklearn import svm  # import model

iris = datasets.load_iris()  # load dataset
clf_svc_cv = svm.SVC(kernel='linear',C=1)  # build a support vector machine with parameters
scores_clf_svc_cv = cross_val_score(clf_svc_cv,iris.data,iris.target,cv=10)  # 10-fold cross validation
print(scores_clf_svc_cv) # print results
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_clf_svc_cv.mean(), scores_clf_svc_cv.std() * 2))  # print accuracy
 ```

https://scikit-learn.org/stable/tutorial/statistical_inference/model_selection.html

You can also use sklearn.model_selection.StratifiedKFold, so that you could iterate 10 folds and perform data preprocessing.
e.g., 

``` 
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
cv = StratifiedKFold(n_splits=10)

for train_idx, test_idx, in cv.split(X, y):
   sm = SMOTE()
   X_train_oversampled, y_train_oversampled = sm.fit_sample(X_train, y_train)
   model = ...  # Choose a model here
   model.fit(X_train_oversampled, y_train_oversampled ) 
   y_pred = model.predict(X_test)
   print(f'For fold {fold}:')
   print(f'Accuracy: {model.score(X_test, y_test)}')
   print(f'f-score: {f1_score(y_test, y_pred)}') 
 ```


In [4]:
#loading the data set and checking the first 5 entries
maind = pd.read_csv("creditcard.csv")
train, test = train_test_split(maind, test_size=0.2)

#separating x and y for testing and training
train_x = train.iloc[:,0:31]
train_y = train['Class'] #note that class is case sensitive
test_x = test.iloc[:,0:31]
test_y = test['Class']
maind.head(5)



,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [5]:
#working with sample code provided to create accuracy measures
iris = datasets.load_iris()  # load dataset
clf_svc_cv = svm.SVC(kernel='linear',C=1)  # build a support vector machine with parameters
scores_clf_svc_cv = cross_val_score(clf_svc_cv,iris.data,iris.target,cv=10)  # 10-fold cross validation
print(scores_clf_svc_cv) # print results
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_clf_svc_cv.mean(), scores_clf_svc_cv.std() * 2))  # print accuracy

[1.         0.93333333 1.         1.         0.86666667 1.
 0.93333333 1.         1.         1.        ]
Accuracy: 0.97 (+/- 0.09)


In [31]:
# separate x and y as values and targets for training and testing data
cv = StratifiedKFold(n_splits=10)


for train_idx, test_idx, in cv.split(train_x, train_y): # what is cv
   sm = SMOTE()
   X_train_oversampled, y_train_oversampled = sm.fit_sample(train_x, train_y)
   model = linear_model.LinearRegression(fit_intercept=False)  # Choose a model here
   model.fit(X_train_oversampled, y_train_oversampled ) 
   y_pred = model.predict(test_x)
   print(f'For fold {fold}:')
   print(f'Accuracy: {model.score(test_x, test_y)}')
   print(f'f-score: {f1_score(test_y, y_pred)}') 

NameError: name 'fold' is not defined

## Task 2: model building and evaluation (40 points). 
Create three models: Logistic Regression, Random Forest, Gradient Boost (XGBoost) using their default hyperparameter values. Report the default parameter values and model performance in terms of precision, recall, F-measure, and AUC. Note that you should perform evalution under the cross validation framework. 

Note that Fraud is your positive class. You don't need to consider all hyperparameters provided by the tool, only focusing on three or four important ones. 

We will apply SMOTE technique to deal with the highly imbalanced dataset. Ref. https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/
E.g.,  
```
sm = SMOTE()
X_train_oversampled, y_train_oversampled = sm.fit_sample(X_train, y_train)
```

How to build random forest and xgboost model? 
E.g.,
```
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)

import xgboost as xgb
gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05).fit(X_train, Y_train)
Y_pred = gbm.predict(X_test)
```




In [8]:
#adjusting the data for oversampling
sm = SMOTE()
x_train_oversampled, y_train_oversampled = sm.fit_sample(train_x, train_y)
print("sample adjustment complete")

sample adjustment complete


In [15]:
#creating logistic regression model
logreg = LogisticRegression()
logreg.fit(x_train_oversampled, y_train_oversampled)
print("built model")
# confusionMatrix = confusion_matrix(test_y, y_pred)
# print(confusionMatrix)

built model
[[54994  1876]
 [   15    77]]


(0.5195769203227807, 0.9019845031765812, 0.5292017172499341, None)

In [29]:
#testing model and getting predictions  
y_pred = logreg.predict(test_x)
#finding precision and recall
from sklearn.metrics import precision_recall_fscore_support as score
# precision, recall, fscore, support = score(test_y, y_pred)
results = score(test_y, y_pred, average='macro')
# Returns list of precision, recall, fscore, support
print("precision, recall, fscore, support")
print(results)
#Finding AUC
from sklearn.metrics import roc_auc_score
AUC = roc_auc_score(test_y, y_pred)
print("AUC")
print(AUC)

precision, recall, fscore, support
(0.5195769203227807, 0.9019845031765812, 0.5292017172499341, None)
AUC
0.9019845031765812


In [17]:
#building the random forest regression model
#Note that running this block takes a long time because the random forest classifier is heavy
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(x_train_oversampled, y_train_oversampled)


(1.0, 1.0, 1.0, None)

In [30]:
# Testing the random forest model and getting precision & recall
y_pred = random_forest.predict(test_x)
#finding precision and recall of Random Forest
from sklearn.metrics import precision_recall_fscore_support as score
# precision, recall, fscore, support = score(test_y, y_pred)
results = score(test_y, y_pred, average='macro')
# Returns list of precision, recall, fscore, support
print("precision, recall, fscore, support")
print(results)
#Finding AUC
from sklearn.metrics import roc_auc_score
AUC = roc_auc_score(test_y, y_pred)
print("AUC")
print(AUC)

precision, recall, fscore, support
(1.0, 1.0, 1.0, None)
AUC
1.0


In [18]:
# Building the XGboost model
gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05).fit(x_train_oversampled, y_train_oversampled)


(1.0, 1.0, 1.0, None)

In [26]:
# testing the model and finding precision & recall
y_pred = gbm.predict(test_x)
#finding precision and recall
from sklearn.metrics import precision_recall_fscore_support as score
# precision, recall, fscore, support = score(test_y, y_pred)
results = score(test_y, y_pred, average='macro')
# Returns list of precision, recall, fscore, support
print("precision, recall, fscore, support")
print(results)
#Finding AUC
from sklearn.metrics import roc_auc_score
AUC = roc_auc_score(test_y, y_pred)
print("AUC")
print(AUC)

precision, recall, fscore, support
(1.0, 1.0, 1.0, None)
AUC
1.0


Answer: Please fill the following table based on your experiment results:

| Classifier | Default Hyperparameter values| Precision | Recall | AUC |

| Logistic regression | (n_estimators=100) | 0.52 | 0.92 | 0.90 |

| Random Forest | n_estimators = 100, creiterion = gini, max_depth = none,  | 1.0 | 1.0 | 1.0 |

| XGboost | (max_depth=3, n_estimators=300, learning_rate=0.05) | 1.0 | 1.0 | 1.0 | 


## Task 3:  hyperparameter tuning (40 points).
For each classifier, turn the hyperparameter using random search and grid search. Report the best performance for each classifier and their running time.
Ref. https://scikit-learn.org/stable/auto_examples/model_selection/plot_randomized_search.html


Answer: Please list your best performance model, hyper parameter values for the best model and running time below:


## Task 4:  take-away message (10 points)
Please summary your findings from the above analysis (at least 2 findings):